In [135]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweet-sentiment-extraction/sample_submission.csv
/kaggle/input/tweet-sentiment-extraction/test.csv
/kaggle/input/tweet-sentiment-extraction/train.csv


In [136]:
os.chdir('/kaggle/input/tweet-sentiment-extraction/')

In [137]:
dataset=pd.read_csv('train.csv')

In [138]:
dataset=dataset.drop(['textID','selected_text'],axis=1)

In [139]:
dataset['text'][:10]

0                                                             I`d have responded, if I were going
1                                                   Sooo SAD I will miss you here in San Diego!!!
2                                                                       my boss is bullying me...
3                                                                  what interview! leave me alone
4                      Sons of ****, why couldn`t they put them on the releases we already bought
5    http://www.dothebouncy.com/smf - some shameless plugging for the best Rangers forum on earth
6                                2am feedings for the baby are fun when he is all smiles and coos
7                                                                                      Soooo high
8                                                                                     Both of you
9                            Journey!? Wow... u just became cooler.  hehe... (is that possible!?)
Name: text, dtype: o

In [140]:
testset=pd.read_csv('test.csv')
testset=testset.drop(['textID'],axis=1)

In [141]:
testset.head()

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -- skyscrapers galore). Good tweeps in China: (SH) (BJ).,positive
2,"Recession hit Veronique Branquinho, she has to quit her company, such a shame!",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive


In [142]:
import re
import string

In [143]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}
from nltk.corpus import stopwords


In [127]:
import nltk

In [146]:
import re

def decontracted(phrase):
    # specific
    phrase=phrase.lower()
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\`t", " not", phrase)
    phrase = re.sub(r"\`re", " are", phrase)
    phrase = re.sub(r"\`s", " is", phrase)
    phrase = re.sub(r"\`d", " would", phrase)
    phrase = re.sub(r"\`ll", " will", phrase)
    phrase = re.sub(r"\`t", " not", phrase)
    phrase = re.sub(r"\`ve", " have", phrase)
    phrase = re.sub(r"\`m", " am", phrase)
    
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
        
    return phrase

In [147]:
dataset['text'][100]

'4am. And Im on the beach. Pretty'

In [148]:
def clean_text(text):
    text = re.sub('\[.*?\]', '', text)
    text=re.sub(r'http\S+', '', text)   
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'[^a-zA-Z ]', '', text)
    return text

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, "R":wordnet.ADV}

def lemmatize_words(text):
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN)) for word, pos in pos_tagged_text])

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [149]:
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :remove_emoji(str(x)))
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :remove_emoticons(x))
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :chat_words_conversion(x))
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :decontracted(str(x)))
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :clean_text(str(x)))
dataset['text']=dataset['text'].str.lower()
dataset['text']=dataset['text'].apply(lambda x :lemmatize_words(x))

dataset['text']=dataset['text'].str.lower()

In [150]:
testset['text']=testset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :remove_emoji(str(x)))
dataset['text']=dataset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :remove_emoticons(x))
dataset['text']=dataset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :chat_words_conversion(x))
dataset['text']=dataset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :decontracted(str(x)))
dataset['text']=dataset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :clean_text(str(x)))
dataset['text']=dataset['text'].str.lower()
testset['text']=testset['text'].apply(lambda x :lemmatize_words(x))
testset['text']=testset['text'].str.lower()

In [151]:
total=pd.concat([dataset,testset])

In [152]:
total['text'][:10]

0                                     i would have respond if i be go
1                          sooo sad i will miss you here in san diego
2                                                  my bos be bully me
3                                       what interview leave me alone
4    son of why could not they put them on the release we already buy
5              some shameless plug for the best ranger forum on earth
6            feeding for the baby be fun when he be all smile and coo
7                                                          soooo high
8                                                         both of you
9              journey wow you just become cool hehe be that possible
Name: text, dtype: object

In [115]:
!pip install fastai2

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [153]:
from fastai2.text.all import *

In [154]:
dls = TextDataLoaders.from_df(total, text_col='text', is_lm=True, bs=128, seq_len=80)
dls.show_batch(max_n=3)

,text,text_
0,xxbos one load of wash put away and another load on amaze how the house do not look any good for all my effort xxbos that be awesome ca not believe they have the poster up already i have not see one over here yet xxbos i know right xxunk girl it show why they should not put young people in competition like this though xxbos sugar oh no i be always here xxbos no luck in goin to forum,one load of wash put away and another load on amaze how the house do not look any good for all my effort xxbos that be awesome ca not believe they have the poster up already i have not see one over here yet xxbos i know right xxunk girl it show why they should not put young people in competition like this though xxbos sugar oh no i be always here xxbos no luck in goin to forum i
1,xxbos walk past the school i go to junior high in a it be be tear down xxbos feel good still cough not move on yet it impossible xxbos look like i get a xxunk macbook xxbos oh no it ok they be come on just now x xxbos ca nt find my shoe hope it as bad a it ll get i m happy xxbos good to have a boyfriend like you xxbos hi x your website soo cool,walk past the school i go to junior high in a it be be tear down xxbos feel good still cough not move on yet it impossible xxbos look like i get a xxunk macbook xxbos oh no it ok they be come on just now x xxbos ca nt find my shoe hope it as bad a it ll get i m happy xxbos good to have a boyfriend like you xxbos hi x your website soo cool i
2,someone xxunk knock at my door xxbos btw xxunk be try to beat his flight control score as well maybe you should follow each other xxbos nope neither of our lady be on today xxbos xxunk grace timberlake have a ring to it xxbos thanks for a great night dear perfectly complete my weekend xxbos it awsome when you know that you know that you know xxbos a xxrep 4 w xxunk when ppl make fun of someone laugh their,xxunk knock at my door xxbos btw xxunk be try to beat his flight control score as well maybe you should follow each other xxbos nope neither of our lady be on today xxbos xxunk grace timberlake have a ring to it xxbos thanks for a great night dear perfectly complete my weekend xxbos it awsome when you know that you know that you know xxbos a xxrep 4 w xxunk when ppl make fun of someone laugh their ass


In [155]:
learn = language_model_learner(dls, AWD_LSTM, drop_mult=0.3, metrics=[accuracy],model_dir="/tmp/model/")

In [156]:
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,time
0,6.617160,6.224939,0.089536,00:33
1,6.215685,5.506111,0.123227,00:33
2,5.760194,5.099595,0.153006,00:32
3,5.427315,4.947785,0.172554,00:33
4,5.211040,4.875422,0.183793,00:32
5,5.069867,4.834860,0.188726,00:33
6,4.975963,4.812214,0.192274,00:32
7,4.914463,4.800784,0.193563,00:32
8,4.879329,4.796320,0.194156,00:32
9,4.856500,4.795552,0.194542,00:32


In [157]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-4)

epoch,train_loss,valid_loss,accuracy,time
0,4.824502,4.771616,0.199249,00:35
1,4.789272,4.723950,0.206554,00:34
2,4.748705,4.687513,0.211035,00:34
3,4.710445,4.663615,0.214735,00:34
4,4.680483,4.651900,0.216053,00:34
5,4.650959,4.637079,0.217396,00:34
6,4.630020,4.630718,0.217852,00:33
7,4.617136,4.627446,0.218587,00:34
8,4.609262,4.626242,0.218294,00:34
9,4.599982,4.625946,0.218464,00:34


In [158]:
learn.save_encoder('finetuned')

In [159]:
TEXT = "I love"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [160]:
print("\n".join(preds))

i love the item up on my head how be the a go to be me i wish i actually live in bed that be what i be come to say say you should have a slight drink off the
i love you finally back to bed be at the moment of my birthday i have see the new weather there x just go to the gym and see the cat make me feel like and your farm be go away


In [169]:
dls_clas = TextDataLoaders.from_df(df=total,train_df=dataset,valid_df=testset, text_col='text', is_lm=False, bs=128, seq_len=80)

In [170]:
dls_clas.show_batch(max_n=3)

,text,category
0,xxbos xxrep 3 a xxrep 3 w i m s xxrep 8 o tire today i do nt feel good i do nt wanna go to work wah but yeah i m goin to work in a few min til closing,negative
1,xxbos u xxrep 4 m i have no idea what i m do but my friend be piss off at me now and i do nt know why she hat me so much i need help britney plz,negative
2,xxbos i feel like an mm xxunk xxrep 3 m but i wo not get one coz i need to look good for when i go to the state week go xxrep 5 a xxrep 4 h,neutral


In [171]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy, model_dir="/tmp/model/")

In [172]:
learn = learn.load_encoder('finetuned')

In [173]:
learn.fit_one_cycle(5, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.017824,0.869887,0.601806,00:36
1,0.988789,0.834555,0.625020,00:37
2,0.995574,0.820147,0.628083,00:37
3,0.970913,0.810596,0.632758,00:37
4,0.946728,0.802661,0.637595,00:38


In [174]:
learn.freeze_to(-2)
learn.fit_one_cycle(5, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.877982,0.764780,0.651459,00:38
1,0.778768,0.710240,0.689989,00:37
2,0.721788,0.680249,0.702886,00:38
3,0.693434,0.658605,0.715460,00:38
4,0.672286,0.659053,0.717556,00:37


In [175]:
learn.freeze_to(-4)
learn.fit_one_cycle(5, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.645192,0.654990,0.726423,00:38
1,0.621503,0.632551,0.737546,00:39
2,0.563069,0.647366,0.731904,00:39
3,0.496103,0.643478,0.740932,00:39
4,0.467811,0.651598,0.739481,00:40


In [176]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.464427,0.649668,0.741254,00:40
1,0.453892,0.673293,0.732710,00:39
2,0.440577,0.674354,0.737869,00:40
3,0.422052,0.690133,0.737546,00:40
4,0.419372,0.681971,0.737546,00:38


In [ ]:
!pwd

In [177]:
learn.save('model1')